In [2]:
import numpy as np
import pandas as pd
import requests
from goodreads import client
from sqlalchemy import create_engine

from config import (api_key, secret_key) #Goodreads API KEY
gc = client.GoodreadsClient(api_key, secret_key)

In [3]:
top = pd.read_csv("Top 100 books - TOP 100 - ALL TIME.csv")
top.head()

,Index,Position,ISBN,Title,Author,Imprint,Publisher Group,Volume,Value,RRP,ASP,Binding,Publ Date,Product Class
0,1,1,9780552149518,"Da Vinci Code,The","Brown, Dan",Corgi Books,Transworld Grp,"4,522,025","£22,857,837.53",£7.99,£5.05,Paperback,1-Mar-04,"F2.1 Crime, Thriller & Adventure"
1,3,2,9780747532743,Harry Potter and the Philosopher's Stone#,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,844,316","£19,853,187.43",£6.99,£5.22,Paperback,26-Jun-97,Y2.1 Children's Fiction
2,7,3,9780747538486,Harry Potter and the Chamber of Secrets#,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,184,492","£16,224,021.98",£6.99,£5.07,Paperback,1-Apr-99,Y2.1 Children's Fiction
3,6,4,9780552150736,Angels and Demons#,"Brown, Dan",Corgi Books,Transworld Grp,"3,096,850","£15,537,324.84",£7.99,£5.05,Paperback,1-Jul-03,"F2.1 Crime, Thriller & Adventure"
4,2,5,9780747551003,Harry Potter and the Order of the Phoenix,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,043,226","£33,925,431.19",£16.99,£11.15,Hardback,21-Jun-03,Y2.1 Children's Fiction


In [4]:
url = "https://www.goodreads.com/book/review_counts.json?isbns="
key = f"&key={api_key}"

In [5]:
first = []
x=0
for isbn in top['ISBN']:
    query = url + str(isbn)+key 
    try:
        response = requests.get(query)
        response_json = response.json()
        book_id = response_json['books'][0]['id']
        reviews_count = response_json['books'][0]['reviews_count']
        book = gc.book(book_id)
        author = book.authors
        pages = book.num_pages
        average_rating = book.average_rating
        ratings_count = book.ratings_count
        title = book.title
        book_dict = {"id":int(book_id),"title":str(title),"author":str(author[0]),"isbn":isbn,"num_pages":int(pages), "average_rating":float(average_rating),
                    "ratings_count":int(ratings_count),"reviews_count":int(reviews_count)}
        first.append(book_dict)
        x+=1
        if(x%10==0):
            print(x)
    except Exception as e:
        book_dict = {"id":int(book_id),"title":str(title),"author":str(author[0]),"isbn":isbn,"num_pages":np.nan, "average_rating":float(average_rating),
                    "ratings_count":int(ratings_count),"reviews_count":int(reviews_count)}
        first.append(book_dict)
        x+=1
        print(x)
        print(title)
        print(e)

10
20
You Are What You Eat: The Plan that Will change Your Life
30
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Dr Atkins New Diet Revolution
36
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Captain Corelli's Mandolin
38
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
The Gruffalo
40
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Notes from a Small Island
48
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
50
Small Island
58
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
60
Guinness World Records 2009 (Guinness World)
68
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
70
80
90
Can You Keep a Secret?
97
int() argument must be a string, a bytes-like object or a number, not 'NoneType'
100


In [89]:
df = pd.DataFrame(first)
df

,id,title,author,isbn,num_pages,average_rating,ratings_count,reviews_count
0,269831,"The Da Vinci Code (Robert Langdon, #2)",Dan Brown,9780552149518,593.0,3.85,11128,16842
1,72193,Harry Potter and the Philosopher's Stone (Harr...,J.K. Rowling,9780747532743,223.0,4.47,35643,46609
2,41643790,Harry Potter and the Chamber of Secrets,J.K. Rowling,9780747538486,251.0,4.43,1354,2249
3,578587,"Angels and Demons (Robert Langdon, #1)",Dan Brown,9780552150736,620.0,3.90,13414,20922
4,444327,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,9780747551003,766.0,4.50,18392,25917
...,...,...,...,...,...,...,...,...
95,70949,"The Amber Spyglass (His Dark Materials, #3)",Philip Pullman,9780439993586,548.0,4.09,5917,8567
96,48757025,Can You Keep a Secret?,Sophie Kinsella,9780552771108,NaN,3.84,98,177
97,42876,Down Under,Bill Bryson,9780552997034,398.0,4.07,4983,7952
98,48928582,A Spot of Bother,Mark Haddon,9780099506928,503.0,3.48,67,162


In [ ]:
#pgAdmin4 
pg_password="zaqxswcde"
pg_user = "postgres"

In [90]:
rds_connection_string = "{pg_user}:{pg_password}@localhost:5432/top_100_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [91]:
df.to_sql(name='goodreads_book', con=engine, if_exists='append', index=False)

In [92]:
# df.to_csv("goodreads_book")